# Music Lyric Generator 
- Make the dataset (20 words input and 1 output) 
- Build a model 
- Train model 
- Test model

## Make the dataset 

In [1]:
import os 
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import numpy
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import ModelCheckpoint


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = []
path = '/content'
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        data.append(f.read())
       
  
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)
        

In [3]:
data

['[Hook]\nI\'ve been down so long, it look like up to me\nThey look up to me\nI got fake people showin\' fake love to me\nStraight up to my face, straight up to my face\nI\'ve been down so long, it look like up to me\nThey look up to me\nI got fake people showin\' fake love to me\nStraight up to my face, straight up to my face [Verse 1]\nSomethin\' ain\'t right when we talkin\'\nSomethin\' ain\'t right when we talkin\'\nLook like you hidin\' your problems\nReally you never was solid\nNo, you can\'t "son" me\nYou won\'t never get to run me\nJust when shit look out of reach\nI reach back like one, three\nLike one, three, yeah [Pre-Hook]\nThat\'s when they smile in my face\nWhole time they wanna take my place\nWhole time they wanna take my place\nWhole time they wanna take my place\nYeah, I know they wanna take my place\nI can tell that love is fake\nI don\'t trust a word you say\nHow you wanna clique up after your mistakes?\nLook you in the face, and it\'s just not the same [Hook]\nI\'ve

In [66]:
print(len(data[0].split()))





def preprocess_data(data):
  new_data = []
  i=0
  for text in data:
    text = text.replace("\n"," ").replace("'","").replace("\t","").replace(",","").replace(".","").replace("[","").replace("-","").replace("]","").replace("?","")
    text_tokens = word_tokenize(text)
    
    text = " ".join([word for word in text_tokens if not word in stopwords.words()])
    print(text)
    
    new_data.append(text)
    
  return new_data
data = preprocess_data(data)


24847
Hook Ive long look like They look I got fake people showin fake love Straight straight Ive long look like They look I got fake people showin fake love Straight straight Verse 1 Somethin aint right talkin Somethin aint right talkin Look like hidin problems Really never solid No cant `` You wont never get run Just shit look reach I reach back like three Like three yeah PreHook Thats smile Whole time wan place Whole time wan place Whole time wan place Yeah I know wan place I tell love fake I dont trust word say How wan clique mistakes Look Hook Ive long look like They look I got fake people showin fake love Straight straight Ive long look like They look I got fake people showin fake love Straight straight Verse 2 Yeah straight tryna play safe Vibe switch like night day I see like right away I came changed I caught whole play Since things never PreHook Thats smile Whole time wan place Whole time wan place Whole time wan place Yeah I know wan place I tell love fake ( I tell love fake 

In [67]:
data[0]

'Hook Ive long look like They look I got fake people showin fake love Straight straight Ive long look like They look I got fake people showin fake love Straight straight Verse 1 Somethin aint right talkin Somethin aint right talkin Look like hidin problems Really never solid No cant `` You wont never get run Just shit look reach I reach back like three Like three yeah PreHook Thats smile Whole time wan place Whole time wan place Whole time wan place Yeah I know wan place I tell love fake I dont trust word say How wan clique mistakes Look Hook Ive long look like They look I got fake people showin fake love Straight straight Ive long look like They look I got fake people showin fake love Straight straight Verse 2 Yeah straight tryna play safe Vibe switch like night day I see like right away I came changed I caught whole play Since things never PreHook Thats smile Whole time wan place Whole time wan place Whole time wan place Yeah I know wan place I tell love fake ( I tell love fake ) I d

In [125]:

def make_your_dataset(data):
  X = []
  Y = []
  sequence_length = 100
  all_words =  " ".join(data)
  data1 = all_words.split()
  word_to_int = {}
  for i,word in enumerate(set(data1)):
    word_to_int[word] = i
  for i in range(0,len(data1)-sequence_length):
    sequence_in = data1[i:i + sequence_length]
    sequence_out = data1[i + sequence_length]

    X.append([word_to_int[char] for char in sequence_in])
    Y.append(word_to_int[sequence_out])
  n_patterns = len(X)
  # reshape the input into a format compatible with LSTM layers
  print(X[0])
  X1 = numpy.reshape(X, (n_patterns, sequence_length, 1))
  # normalize input
  X1 = X1 / float(n_patterns)
  Y = to_categorical(Y)
  return X1,Y,word_to_int,n_patterns,X

  



In [126]:
X_data,Y_data, vocab,n_vocab,network_input = make_your_dataset(data)

[2068, 230, 2357, 1678, 2239, 2034, 1678, 2668, 2106, 2192, 2820, 3496, 2192, 3309, 366, 1513, 230, 2357, 1678, 2239, 2034, 1678, 2668, 2106, 2192, 2820, 3496, 2192, 3309, 366, 1513, 902, 4216, 1958, 2953, 3319, 3204, 1958, 2953, 3319, 3204, 1538, 2239, 3939, 3620, 165, 1857, 1106, 2853, 2435, 4004, 3286, 1347, 1857, 2103, 4129, 2821, 174, 1678, 1291, 2668, 1291, 3763, 2239, 306, 140, 306, 1567, 4016, 1380, 3106, 1896, 1515, 1649, 2841, 1896, 1515, 1649, 2841, 1896, 1515, 1649, 2841, 1292, 2668, 2387, 1649, 2841, 2668, 1090, 3309, 2192, 2668, 479, 124, 964, 2238, 1025, 1649, 695]


In [96]:
len(vocab)

4226

In [97]:
X_data.shape,Y_data.shape

((24743, 100, 1), (24743, 4226))

In [98]:
model = Sequential()


model.add(LSTM(
    512,
    input_shape=(X_data.shape[1], X_data.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(Y_data.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [99]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
dropout_18 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 100, 512)          2099200   
_________________________________________________________________
dropout_19 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_20 (Dropout)         (None, 256)              

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X_data, Y_data,epochs=100, callbacks=callbacks_list)

Epoch 1/100
774/774 [==============================] - 43s 52ms/step - loss: 6.9807
Epoch 2/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7875
Epoch 3/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7700
Epoch 4/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7599
Epoch 5/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7576
Epoch 6/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7546
Epoch 7/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7514
Epoch 8/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7501
Epoch 9/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7487
Epoch 10/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7484
Epoch 11/100
774/774 [==============================] - 39s 51ms/step - loss: 6.7493
Epoch 12/100
774/774 [==============================] - 39s 51ms/step - lo

In [132]:
def predict():
  start = numpy.random.randint(0, len(X_data)-1)

  pattern = network_input[start]

  prediction_output = []
  inverted_int_word = {}
  for i,j in vocab.items():
    inverted_int_word[j] = i
  # generate 500 notes
  x = 0
  for note_index in range(500):

      prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
      prediction_input = prediction_input / float(n_vocab)
      prediction = model.predict(prediction_input, verbose=0)
      index = numpy.argmax(prediction)
    
      result = inverted_int_word[index]
  
      prediction_output.append(result)
      pattern.append(index)
      pattern = pattern[1:len(pattern)]
      x += 1
  return " ".join(prediction_output)

In [134]:
predict()

'weed weed weed weed weed weed weed weed weed weed weed weed system system system system weed weed weed weed system system system system system system system weed weed system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system system